In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from os.path import join
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [2]:
print(tf.__version__)

2.7.0


In [3]:
# Data load 
datax = np.load(join('data', 'MNIST.npy'))
datay = np.load(join('data', 'Label.npy'))

In [4]:
datax_reshape = datax.reshape([datax.shape[0], datax.shape[1]*datax.shape[2]])
datax_reshape_norm = datax_reshape/255
print('최대 : {}, 최소 : {}'.format(np.max(datax_reshape_norm), np.min(datax_reshape_norm)))

최대 : 1.0, 최소 : 0.0


In [7]:
datax_reshape_norm = datax_reshape_norm.astype('float32')

In [8]:
datay_onehot = to_categorical(datay)
print(datay[0:10])
print(datay_onehot[0:10,:])

[5 0 4 1 9 2 1 3 1 4]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [9]:
# Train, Test Data Split
from sklearn.model_selection import train_test_split
trnx, tstx, trny, tsty = train_test_split(datax_reshape_norm, datay_onehot, test_size = 0.3, random_state =111)

In [14]:
initializer = tf.initializers.glorot_uniform()
def get_weight(shape,name):
    return tf.Variable(initializer( shape ), name=name, trainable=True, dtype=tf.float32)

shapes = [
    [trnx.shape[1], 1000], 
    [1000,2000] ,
    [2000,1000] ,
    [1000, trny.shape[1]] ,
]

weights = [] 

for i in range(len(shapes)):
    weights.append(get_weight(shapes[i], 'weight{}'.format(i)))



In [15]:
def model(X, dropout_rate):
    h1 = tf.nn.relu(tf.matmul(X,weights[0]))
    h1 = tf.nn.dropout(h1, dropout_rate)
    
    h2 = tf.nn.relu(tf.matmul(h1,weights[1]))
    h2 = tf.nn.dropout(h2, dropout_rate)
    
    h3 = tf.matmul(h2, weights[2])
    h3 = tf.nn.relu(h3)
    h3 = tf.nn.dropout(h3, dropout_rate)
    
    out = tf.nn.softmax(tf.matmul(h3,weights[3]))
    
    return out

In [17]:
dropout_rate =0.5
learning_rate =0.01
optimizer = tf.optimizers.Adagrad(learning_rate)
def loss(target, pred):
    return tf.losses.categorical_crossentropy(target,pred)

def train_step(model, inputs, outputs):
    with tf.GradientTape() as tape:
        current_loss = loss(outputs,model(inputs,dropout_rate))
        grads = tape.gradient(current_loss,weights)
        optimizers.apply_gradients(zip(grads,weights))
        print(tf.reduce_mean(current_loss))

In [ ]:
batch_size = 100
max_epoch = 100
for i in range(max_epoch):
    idx = np.random.permutation(len(trnx))[0:batch_size]
    train_step(model, trnx[idx, :], trny[idx,:])

In [ ]:
from sklearn.metrics import accuracy_score

pred = model(tstx,0)
print(tf.argmax(tsty, axis =1))
print(tf.argmax(pred, axis =1))

accuracy_score(tf.argmax(tsty, axis = 1), tf.argmax(pred, axis = 1))